In [ ]:
%pylab inline
from numpy.polynomial.legendre import leggauss
from scipy.special import legendre

In [ ]:
Nq = 64
x, wq = leggauss(Nq)
Nenr = 5 # total enrichment
Np = 4 # polynomial enrichment using Legendre polynomials
P = empty((Nenr,Nq), dtype="double")
for n in range(Np):
    P[n,:] = legendre(n)(x)*sqrt(n+1./2)

# Non-polynomial enrichment
f = sin(x)
f = f / sqrt(sum(f**2*wq))
P[Nenr-1,:] = f

In [ ]:
def getS(B):
    "Calculate an overlap matrix"
    N = size(B,0)
    S = empty((N,N), dtype="double")
    for i in range(N):
        for j in range(N):
            S[i,j] = sum(B[i,:]*B[j,:]*wq)
    return S

In [ ]:
S = getS(P)
eigs, vecs = eigh(S)

The matrix **S** has ones on the diagonal, because all enrichment (polynomial and non-polynomial) is normalized to 1:

In [ ]:
S.diagonal()

However, while the Legendre polynomial part of the matrix is a unit matrix, the non-polynomial part is not:

In [ ]:
print array_str(S, precision=3, suppress_small=True)

Now we use the Löwdin symmetric orthogonalization to make everything orthogonal:

In [ ]:
P2 = empty((Nenr,Nq), dtype="double")
for i in range(Nenr):
    v = 0
    for j in range(Nenr):
        v += vecs[j,i]*P[j,:]
    P2[i,:] = v / sqrt(eigs[i])

The overlap of the new orthonormal basis P2 is a unit matrix:

In [ ]:
S2 = getS(P2)
print array_str(S2, precision=3, suppress_small=True)

In [ ]:
figure(figsize(16, 4))
subplot(121)
title("Original basis")
for n in range(Nenr):
    plot(x, P[n,:])
subplot(122)
title("Orthonormal basis")
for n in range(Nenr):
    plot(x, P2[n,:])

Now we look at the eigenvalues of the overlap matrix **S**:

In [ ]:
eigs

And we remove the smallest eigenvalue, which is close to zero, and construct the new basis:

In [ ]:
P3 = empty((Nenr-1,Nq), dtype="double")
for i in range(Nenr-1):
    v = 0
    for j in range(Nenr):
        v += vecs[j,i+1]*P[j,:]
    P3[i,:] = v / sqrt(eigs[i+1])

The new overlap matrix is again a unit matrix, but this time we have removed one DOF:

In [ ]:
S3 = getS(P3)
print array_str(S3, precision=3, suppress_small=True)

In [ ]:
figure(figsize(16, 8))
subplot(221)
title("Original Basis")
for n in range(Nenr):
    plot(x, P[n,:])
subplot(222)
title("Orthonormal Basis")
for n in range(Nenr):
    plot(x, P2[n,:])
subplot(223)
title("Reduced Orthonormal Basis")
next(gca()._get_lines.prop_cycler)
for n in range(Nenr-1):
    plot(x, P3[n,:])

The condition number of all the overlap matrices:

In [ ]:
print cond(S)
print cond(S2)
print cond(S3)